<a href="https://colab.research.google.com/github/chema74/AI-Portfolio-2025/blob/main/08_Feature_Selection_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📈 Módulo 8: Selección de Características y Pipelines

La **Selección de Características (Feature Selection)** y la **Ingeniería de Características (Feature Engineering)** son procesos que buscan optimizar el conjunto de datos de entrada ($X$) para mejorar la precisión y reducir el tiempo de entrenamiento del modelo.

## Conceptos Clave

### 1. La Maldición de la Dimensionalidad (Curse of Dimensionality)
* **Definición:** A medida que el número de características (dimensiones) aumenta, la cantidad de datos requerida para mantener la misma densidad de información crece exponencialmente.
* **Problema:** Un número excesivo de características irrelevantes o redundantes puede hacer que los algoritmos de ML funcionen peor, sean más lentos y requieran más recursos.

### 2. Métodos de Selección de Características

La selección consiste en eliminar las características menos informativas:
* **Métodos de Filtro (Filter Methods):** Usan estadísticas (como correlación o puntuación chi-cuadrado) para evaluar la relación de cada característica con la variable objetivo, *independientemente del modelo*. Son rápidos.
* **Métodos de Envoltura (Wrapper Methods):** Usan el rendimiento del modelo (ej. precisión) para evaluar subconjuntos de características. Son más lentos, pero a menudo más precisos (ej. Eliminación Recursiva de Características - RFE).

### 3. El Pipeline de Scikit-learn (Flujo de Trabajo)
Un **Pipeline** es una herramienta de `sklearn` que encadena pasos de preprocesamiento (como el escalado de datos) y un estimador (el modelo de ML) en un solo objeto.
* **Ventaja:** Garantiza que los pasos de preprocesamiento (ej. `StandardScaler` o `fit`) se apliquen consistentemente a los datos de entrenamiento y de prueba, **evitando la fuga de datos (data leakage)**.

En este ejercicio, aplicaremos la Eliminación Recursiva de Características (RFE), un método de envoltura, y lo implementaremos dentro de un Pipeline para mantener un flujo de trabajo limpio y profesional.

In [1]:
# =======================================================
# PASO 1: Importar librerías
# =======================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE # Importamos la Eliminación Recursiva de Características
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline # Importamos la herramienta Pipeline
from sklearn.metrics import accuracy_score

print("✅ Librerías de Selección de Características y Pipeline importadas.")
print("-" * 70)

# =======================================================
# PASO 2: Preparación de Datos con Múltiples Características
# =======================================================

# Usaremos un DataFrame con más variables predictoras (X)
datos = {
    'Precio_Unitario': [15.5, 15.5, 15.5, 22.0, 10.0, 15.5],
    'Dias_Envio': [5, 2, 5, 7, 1, 2],
    'Antiguedad_Cliente': [10, 2, 1, 5, 8, 3], # Nueva Característica 1
    'Descuento': [0, 1, 0, 1, 0, 0], # Nueva Característica 2 (binaria)
    'Cantidad': [3, 1, 3, 2, 4, 1]
}
df = pd.DataFrame(datos)

# Creamos la variable objetivo binaria (Y) como en el Módulo 5 (Clasificación)
media_cantidad = df['Cantidad'].mean()
df['Cantidad_Alta'] = np.where(df['Cantidad'] > media_cantidad, 1, 0)

# X: Seleccionamos TODAS las características para la selección
X = df[['Precio_Unitario', 'Dias_Envio', 'Antiguedad_Cliente', 'Descuento']]
Y = df['Cantidad_Alta']

print(f"1. Total de características iniciales (dimensionalidad): {X.shape[1]}")
print("-" * 70)


# =======================================================
# PASO 3: División de Datos
# =======================================================
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42 # Cambiamos el test_size para tener más datos de prueba
)

print(f"2. Muestras de entrenamiento: {len(X_train)}")
print(f"3. Muestras de prueba: {len(X_test)}")
print("-" * 70)


# =======================================================
# PASO 4: Implementación del Pipeline (Escalado + Selección RFE + Modelo)
# =======================================================

# 4a. Definimos el Estimador (Modelo Base) para RFE
# Usaremos Regresión Logística como base para evaluar las características
estimator = LogisticRegression(solver='liblinear', random_state=42)

# 4b. Definimos el paso de Selección de Características (RFE)
# RFE usará el Estimador (RegLog) y elegirá las 2 mejores características de las 4.
selector_rfe = RFE(estimator, n_features_to_select=2, step=1)

# 4c. Creamos el Pipeline: Encadenamos Escalado, RFE y el Modelo final
modelo_pipeline = Pipeline(steps=[
    ('escalador', StandardScaler()), # Paso 1: Escalar las variables (vital para LogReg/SVM)
    ('seleccion', selector_rfe),     # Paso 2: Seleccionar las 2 mejores variables
    ('clasificador', estimator)      # Paso 3: Entrenar el modelo final
])

# 4d. Entrenar el Pipeline (Aplica todos los pasos en orden, solo a los datos de train)
modelo_pipeline.fit(X_train, Y_train)

print("4. Pipeline entrenado (Escalado, RFE y Logística aplicados).")

# 4e. Identificar las características seleccionadas por RFE
# Debemos acceder al paso 'seleccion' del pipeline y luego al selector:
columnas_seleccionadas = X.columns[modelo_pipeline.named_steps['seleccion'].support_]

print(f"5. Características seleccionadas por RFE: {list(columnas_seleccionadas)}")
print("-" * 70)


# =======================================================
# PASO 5: Predicción y Evaluación
# =======================================================

# El pipeline predice aplicando automáticamente el escalado y la selección a X_test
Y_pred = modelo_pipeline.predict(X_test)
precision = accuracy_score(Y_test, Y_pred)

print("6. Evaluación del rendimiento en datos de PRUEBA:")
print(f"   Precisión (Accuracy) con Pipeline y RFE: {precision:.4f}")
print("-" * 70)

✅ Librerías de Selección de Características y Pipeline importadas.
----------------------------------------------------------------------
1. Total de características iniciales (dimensionalidad): 4
----------------------------------------------------------------------
2. Muestras de entrenamiento: 4
3. Muestras de prueba: 2
----------------------------------------------------------------------
4. Pipeline entrenado (Escalado, RFE y Logística aplicados).
5. Características seleccionadas por RFE: ['Precio_Unitario', 'Descuento']
----------------------------------------------------------------------
6. Evaluación del rendimiento en datos de PRUEBA:
   Precisión (Accuracy) con Pipeline y RFE: 1.0000
----------------------------------------------------------------------
